# Select Your IPTS 

In [1]:
from __code.normalization_new_ui import *

from __code import system
system.System.select_working_dir()
from __code.__all import custom_style
custom_style.style()

** Using Debugging Mode! **


HBox(children=(Label(value='Select Working Folder', layout=Layout(width='20%')), Select(index=8, options=('BraggEdge', 'Brianne', 'DOE_letters', 'FITSimages', 'IPTS-12534', 'IPTS_13639_resonance_shawn', 'IPTS_14832', 'IPTS_16212', 'IPTS_16259', 'IPTS_16475', 'IPTS_16893', 'IPTS_17099', 'IPTS_17685', 'IPTS_17981', 'IPTS_18806', 'IPTS_19037', 'IPTS_19558', 'IPTS_19621', 'IPTS_19626', 'IPTS_test_normalization', 'IPTS_testing_time_spectra', 'ImagesToRegisterForGian', 'Indu', 'Keita_code', 'Pour_ibeatles_video', 'ResonanceImaging', 'SENSITIVE', 'SarmaGUI', 'SphereCoordinates3D', 'TempData', 'gian', 'iBeatles_debugging'), value='IPTS_16259')))

# Description 

This notebook performs the normalization of a set of images (tiff or fits) by OBs. 

The steps are:
 - select your **sample images**
 - select your **ob images**
 - select your **df images** (optional)
 - all the data will be loaded
 - preview of the sample and ob
 - selection of a **background** region in the sample data
 - normalization is performed
 - export of the normalized data

# Python Import

In [2]:
%matplotlib notebook

# Loading Images 

In [3]:
files = Files()
sample_panel = SampleSelectionPanel(working_dir=system.System.get_working_dir())
sample_panel.init_ui(files=files)
wizard = WizardPanel(sample_panel=sample_panel)

HTML(value='Please wait...')

Label(value='Selection of All Input Files', layout=Layout(border='1px lighgray solide', height='35px', padding='8px', width='300px'))

VBox(children=(HBox(children=(Label(value='Instructions:', layout=Layout(width='20%')), Label(value='Select list of Sample files (mandatory) ', layout=Layout(width='50%')))), HBox(children=(Label(value='Sample selected:', layout=Layout(width='20%')), Label(value='0 files selected', layout=Layout(width='50%'))))), layout=Layout(border='1px lightgray solid', margin='5px', padding='15px'))

HBox(children=(Label(value='         ', layout=Layout(width='70%')), Button(button_style='warning', description='Next Step>>', disabled=True, layout=Layout(width='20%'), style=ButtonStyle(), tooltip='Click to move to next step')))

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19621', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store     |     Thu Dec 14 13:58:29 2017', ' CT            |     Directory', ' DF            |     Directory', ' OB            |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

...

HTML(value='Please wait...')

VBox(children=(Label(value='', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19621/DF', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' 20170811_DF_0030_0005.tiff     |     Thu Dec 14 13:55:56 2017'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=1)))

HBox(children=(Label(value='Loading ob', layout=Layout(width='20%')), IntProgress(value=0, max=1)))

HBox(children=(Label(value='Loading df', layout=Layout(width='20%')), IntProgress(value=0, max=1)))

# Preview Data

## sample

In [4]:
o_norm = sample_panel.o_norm
o_norm.plot_images(data_type='sample')

interactive(children=(IntSlider(value=0, continuous_update=False, description='sample Index', max=0), Output()), _dom_classes=('widget-interact',))

## ob

In [5]:
o_norm.plot_images(data_type='ob')

interactive(children=(IntSlider(value=0, continuous_update=False, description='ob Index', max=0), Output()), _dom_classes=('widget-interact',))

# Select Background Region 

In [6]:
o_norm.with_or_without_roi()

VBox(children=(Label(value='Do you want to select a region of interest (ROI) that will make sure that the sample background matches the OB background'), HBox(children=(Label(value='With or Without ROI?'), RadioButtons(layout=Layout(width='50%'), options=('yes', 'no'), value='yes')))))

In [7]:
o_norm.select_sample_roi()

Label(value='-> You chose not to select any ROI! Next step: Normalization')

# Normalization

In [9]:
o_norm.run_normalization()

HBox(children=(Label(value='Normalization', layout=Layout(width='20%')), IntProgress(value=0, max=1)))

# Export 

In [ ]:
o_norm.select_export_folder()

In [ ]:
o_norm.export()